<a href="https://colab.research.google.com/github/aso1901102/AI_tech2020/blob/master/2020AI0404_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Doc2Vecで文章を学習したコーパスモデルを作って保存

### データリストの作成

In [1]:
#学習対象とする青空文庫の作品リスト --- (*1)
list = [
    {"author":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
    ]},
    {"author":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"},
    ]},
    {"author":{
        "name":"ポー エドガー・アラン",
        "url":"https://www.aozora.gr.jp/cards/000094/files/"}, 
     "books":[
        {"name":"ウィリアム・ウィルスン","zipname":"2523_ruby_19896.zip"},
        {"name":"落穴と振子","zipname":"1871_ruby_17551.zip"},
        {"name":"黒猫","zipname":"530_ruby_20931.zip"},
        {"name":"群集の人","zipname":"56535_ruby_69925.zip"},
        {"name":"沈黙","zipname":"56537_ruby_70425.zip"},
    ]},
    {"author":{
        "name":"紫式部",
        "url":"https://www.aozora.gr.jp/cards/000052/files/"}, 
     "books":[
        {"name":"源氏物語 01 桐壺","zipname":"5016_ruby_9746.zip"},
        {"name":"源氏物語 02 帚木","zipname":"5017_ruby_9752.zip"},
        {"name":"源氏物語 03 空蝉","zipname":"5018_ruby_9754.zip"},
        {"name":"源氏物語 04 夕顔","zipname":"5019_ruby_9761.zip"},
        {"name":"源氏物語 05 若紫","zipname":"5020_ruby_11253.zip"},
    ]},
]

### MeCabのインストール

In [2]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!pip install mecab-python3 > /dev/null

# MeCabの実行時の指定パスをインストールパスにリンクさせる
# シンボリックリンク（/etc/mecabrcを/usr/local/etc/mecabrcで参照できるようにする）
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

### データリストを検索して順番に返す関数：book_listを定義

In [24]:
# 作品リスト（list）をループ
def book_list():
  for novellist in list:
    author = novellist["author"]
    for book in author:
      # yieldで順次値をリターンして
      yield author, book
    # for ここまで
  # for ここまで
# def ここまで

### 指定された書籍名（book）のZipファイルを展開し、ファイルのテキストデータを読み込んでリターンする関数：read_book

In [36]:
import zipfile
import os.path
import urllib.request as req

# zipファイルを開き、なかの文書データを取得
def read_book(author, book):
  zipname = book["zipname"]
  print(zipname)
  # zipファイルがなければDL
  if not os.path.exists(zipname):
    req.urlretrieve(author["url"] + zipname, zipname)
    zipname = book["zipname"]

  # zipファイルを展開
  with zipfile.ZipFile(zipname, "r") as zf:
    # Zipファイルに含まれるファイルを開く
    for filename in zf.namelist():
      # txt拡張子以外は処理スキップ
      if os.path.splittext(filename)[1] != ".txt":
        continue
      with zf.open(filename, "r") as f:
        # 読み込むファイルはShift-JISなので文字コードを指定してデコード
        return f.read().decode("shift-jis")

### MeCabインスタンス生成と引数で渡された文章データを形態素解析した配列にリターンする関数：split_word

In [9]:
import MeCab

# MeCabインスタンス生成
mecab = MeCab.Tagger()

# 引数テキストを形態素解析　＆　ストップワード除去して配列
def split_word(text):
  node = mecab.parseToNode(text)

  # 分かち書き（形態素解析　＆　ストップワード除去）して結果配列
  wakati_words = []
  while node is not None:
    hinshi = node.feature.split(",")[0]
    if hinshi in ["名詞"]:
      wakati_words.append(node.surface)
    elif hinshi in ["動詞", "形容詞"]:
      wakati_words.append(node.feature.split(",")[6])
    node = node.next
  # while ここまで
  return watati_words  

## 学習モデルのメイン処理

### 作品リストをループしながらZipファイルを開き、テキストデータを形態素解析
### タグ（ラベル）付きドキュメントとして学習させる

In [39]:
from gensim import models
from gensim.models.doc2vec import TaggedDocument

# 作品リストをDoc2Vecが学習で消えるTaggdDocument（説明変数＆目的変数）にして配列に追加
documents = []

# 作品リストをループ
for author, book in book_list():
  # 作品の文字列文章データを取得
  words = read_book(author, book)
  # 作品配列を分かち書き
  wakati_words = sprit_words(words)

  # TaggedDocmumentの作成(説明変数：分かち書きにした作品　タグ：作者/作品名  )
  document = TaggedDocmument(wakati_words,[author["name"] + ":" + book["name"]])
  
  # 登録するタグドキュメント
  print(document)

  # 配列に登録
  documents.append(document)
  # for　ここまで
# TaggedDocumentの配列を使ってDoc2Vecの学習モデルを作成
model = models.Doc2Vec(documents, dm=0, vector_size=300, window=15, min_count=1)

# 作成した学習モデルをファイル保存
model.save("aozora.model")
print("モデル保存完了")

{'name': '宮澤 賢治', 'url': 'https://www.aozora.gr.jp/cards/000081/files/'}


TypeError: ignored

## Doc2Vecで学習したコーパスモデルを使って関連度の高い文章を推論

### 保存されたコーパスモデルを読み込む

In [27]:
from gensim import models

# 保存したDoc2Vec学習ファイルを読み込む
model = models.Doc2Vec.load("aozora.model")

### 指定されたZipファイルをDLして、テキストデータをリターンする関数：read_book

In [30]:
import urllib.request as req
import zipfile
import os.path

# 分類用のZipファイルを開いて文章データを取得
def read_book(url, zipname):
  if not os.path.exists(zipname):
    req.urlretrieve(url, zipname)

  # 指定したファイルを開いてリターン
  with zipfile.ZipFile(zipname, "r") as zf:
    for filename in zf.namlist():
      with zf.open(filename, "r") as f:
        return f.read().decode("shift-jis")

### 引き渡されたタイトルとURLを読み込んで、Doc2Vecのが学友モデルから関連度の高いデータを推論させる関数：similar

In [31]:
# 引数のタイトル、URLの作品を分類する
def similar(title, url):
  zipname = url.split("/")[-1]

  words = read_book(url, zipname)
  wakati_words = split_words(words)

  # 文章ベクトルを取得
  vector = nodel.infer_vector(wakati_words)
  print("---[", title, "]と似た作品は？---")

  # 推論結果
  print(model.docvecs.most_similar([vector], topn=3))
  print("")

### Vecのコーパスモデルによる推論

In [32]:
# メイン実行処理

# 引き渡したデータと関連度の高いデータ
similar("宮沢 賢治:よだかの星", 
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")
 
similar("芥川 龍之介:犬と笛", 
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")
 
similar("ポー エドガー・アラン:マリー・ロジェエの怪事件", 
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")
 
similar("紫式部:源氏物語 06 末摘花", 
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

AttributeError: ignored